In [1]:
import json
import os
import requests
from dotenv import load_dotenv
import pandas as pd

In [2]:
load_dotenv()
base_url = os.environ.get('N8N_BASE_URL')
fuel_path = os.environ.get('WEBHOOK_PATH_FUEL')
n8n_token = os.environ.get('N8N_WEBHOOK_TOKEN')

In [3]:
with open('./raw/data_1.json', 'r') as f:
  data = json.load(f)
df = pd.json_normalize(data)
df.drop(['period.start'], axis=1, inplace=True)
df.rename(columns={'period.end': 'date'}, inplace=True)
df['date'] = df['date'].astype('str')
zero_mileage = 212812
df['mfz'] = df['distance_km'].cumsum() - df['distance_km']
df['mileage'] = zero_mileage - df['mfz']
df['ac'] = df['distance_km'] / df['fuel_liters']
df.head()

,days,fuel_liters,distance_km,avg_consumption_l_per_100km,date,mfz,mileage,ac
0,25,33.29,456,7.3,2025-12-21,0,212812,13.697807
1,31,27.83,411,6.8,2025-11-26,456,212356,14.768236
2,12,19.85,278,7.1,2025-10-26,867,211945,14.005038
3,16,34.31,477,7.2,2025-10-14,1145,211667,13.902652
4,18,40.61,647,6.3,2025-09-28,1622,211190,15.932036


In [4]:
odo = pd.read_csv('./raw/data_1.csv')
odo['date'] = odo['date'].astype('str')
df = df.join(odo.set_index('date'), on='date')
df.head()

,days,fuel_liters,distance_km,avg_consumption_l_per_100km,date,mfz,mileage,ac,year_month,amount_rub,odometer_km
0,25,33.29,456,7.3,2025-12-21,0,212812,13.697807,2025-12,2310.99,212812
1,31,27.83,411,6.8,2025-11-26,456,212356,14.768236,2025-11,1851.81,212356
2,12,19.85,278,7.1,2025-10-26,867,211945,14.005038,2025-10,1320.82,211945
3,16,34.31,477,7.2,2025-10-14,1145,211667,13.902652,2025-10,2282.99,211667
4,18,40.61,647,6.3,2025-09-28,1622,211190,15.932036,2025-09,2702.19,211190


In [5]:
df['price_per_liter'] = round(df['amount_rub'] / df['fuel_liters'], 2)
df.head()

,days,fuel_liters,distance_km,avg_consumption_l_per_100km,date,mfz,mileage,ac,year_month,amount_rub,odometer_km,price_per_liter
0,25,33.29,456,7.3,2025-12-21,0,212812,13.697807,2025-12,2310.99,212812,69.42
1,31,27.83,411,6.8,2025-11-26,456,212356,14.768236,2025-11,1851.81,212356,66.54
2,12,19.85,278,7.1,2025-10-26,867,211945,14.005038,2025-10,1320.82,211945,66.54
3,16,34.31,477,7.2,2025-10-14,1145,211667,13.902652,2025-10,2282.99,211667,66.54
4,18,40.61,647,6.3,2025-09-28,1622,211190,15.932036,2025-09,2702.19,211190,66.54


In [9]:
for _, row in df[::-1].iterrows():
  data = {
    'mileage': row['mileage'],
    'fuel_added': row['fuel_liters'],
    'price_per_liter': 0,
    'is_full': True,
    'refuel_date': row['date'],
    'vehicle_id': 'fabia_p620cp47',
    'fuel_grade': '95+',
    'price_per_liter': row['price_per_liter']
  }
  print(data['mileage'])
  r = requests.request(
    method='POST',
    url=f'{base_url}{fuel_path}',
    headers={'X-Auth-Token': n8n_token},
    json=data
  )
  print(r.status_code, r.text)

200621
200 {"id":71}
200964
200 {"id":72}
201361
200 {"id":73}
201683
200 {"id":74}
202232
200 {"id":75}
202600
200 {"id":76}
202902
200 {"id":77}
203149
200 {"id":78}
203691
200 {"id":79}
204223
200 {"id":80}
204656
200 {"id":81}
205154
200 {"id":82}
205815
200 {"id":83}
206350
200 {"id":84}
206987
200 {"id":85}
207451
200 {"id":86}
207948
200 {"id":87}
208622
200 {"id":88}
209058
200 {"id":89}
209340
200 {"id":90}
210000
200 {"id":91}
210543
200 {"id":92}
211190
200 {"id":93}
211667
200 {"id":94}
211945
200 {"id":95}
212356
200 {"id":96}
212812
200 {"id":97}
